In [1]:
## Receive the signal
include("rxsignal_withchannelandfreqoff.jl");

In [2]:
## Load the template signal
include("pss2.jl");

In [3]:
using FFTW

In [4]:
## Load the useful math operations
include("operations.jl");

In [5]:
## Assign the received signal a variable Ŝᵣₓ³
𝑅ₚₛₛ³ = rxs3; # RX Received Signal 3 File Handle
@show size(𝑅ₚₛₛ³); 

size(𝑅ₚₛₛ³) = (616447, 1)


In [6]:
using LinearAlgebra

In [7]:
## Assign the template signal a variable 𝐻ₚₛₛ²
## convert the signal in time domain
Pₚₛₛ² = pss_2; # File Handle
Hₚₛₛ²ᵀ = transpose.(Pₚₛₛ²); # Hessian Transpose ?
𝐻ₚₛₛ² = Fᴵ(Hₚₛₛ²ᵀ); # S Slanted (fourier transform) in time domain
𝐻ₚₛₛ² ./= norm(𝐻ₚₛₛ²);
𝐻ₚₛₛ² = 𝐻ₚₛₛ²[(end-143):end] ⧺ 𝐻ₚₛₛ² ; @show size(𝐻ₚₛₛ²); # end is 2048 in this case, concat math 

size(𝐻ₚₛₛ²) = (2192,)


In [8]:
## Prepare the template signal for convolution
𝐻̅ₚₛₛ² = ⦰(𝐻ₚₛₛ²); # reverse #typed H\overbar
𝐻̅ₚₛₛ²ᴴ = conj(𝐻̅ₚₛₛ²); # conjugate 

In [9]:
using DSP

In [10]:
## Perform the convolution between the 2 signals
𝐻̂ₚₛₛ² = 10 * log10.(abs.( 𝑅ₚₛₛ³ ⊗ 𝐻̅ₚₛₛ²ᴴ )); @show size( 𝐻̂ₚₛₛ² );  #typed \itH\hat

size(𝐻̂ₚₛₛ²) = (618638, 1)


In [11]:
# Find maximum value and its index
Ĉᵩ², 𝑁̂𝑓² = argmax(𝐻̂ₚₛₛ²); @show Ĉᵩ², 𝑁̂𝑓²;

(Ĉᵩ², 𝑁̂𝑓²) = (50.649042476081405, CartesianIndex(6628, 1))


In [12]:
θ₁ = 144
θ₂ = 2048
getindex(𝑁̂𝑓², 1) - ( θ₂ + θ₁ )

4436

In [13]:
using Plots

In [14]:
# Plot the result
m2_chan_plot = plot(𝐻̂ₚₛₛ², xlabel="Sample", ylabel="Power (dB)", title="Convolution Result", ylim=(-20, 60))
savefig(m2_chan_plot,"images/m2_chan_plot.png");

<img src=images/m2_chan_plot.png width='' heigth='' > </img>

In [19]:
########################################################
# Frequency Offset Estimator Function
########################################################
function freq_offset_est(𝑅ₚₛₛ, 𝐻ₚₛₛ, Nf, m, 𝑓ₛ)

    # Frequency offset estimator
    Y = zeros(ComplexF64, length(m));
    L = length(𝐻ₚₛₛ)
    t = 0:(1/𝑓ₛ):((L-1)/𝑓ₛ)

    signal_part = 𝑅ₚₛₛ[Nf:(Nf + L -1)]; @show size( signal_part )
    for j = 1:length(m)
        Y[j] = Y[j] + abs.(∑(exp.(-2*pi*im*m[j].*t) .* conj(𝐻ₚₛₛ) .* ⦰(signal_part)))^2;
    end

    return Y
end

freq_offset_est (generic function with 1 method)

In [20]:
𝑓ₛ = 61.44e6

Δ𝑓 = 10.0
m = -7500.:Δ𝑓:7500.

Nf = getindex(𝑁̂𝑓², 1) - length(𝐻ₚₛₛ²) + 1; @show Nf;

Nf = 4437


In [21]:
# Grab the function Profs Frequency Offset with his values
Y = freq_offset_est(𝑅ₚₛₛ³, 𝐻ₚₛₛ², Nf, m, 𝑓ₛ); # @show Y

size(signal_part) = (2192,)


In [22]:
A_fo, fo = argmax(abs.(Y)); 
@show (10 * log10(A_fo));
@show (fo);

10 * log10(A_fo) = 88.46888484427753
fo = 587


In [23]:
cfo_estim_plot = 
plot(10 * log10.(abs.(Y)), xlabel="Sample", ylabel="Power (dB)", title="Estimation Result")
savefig(cfo_estim_plot,"images/cfo_estim_plot.png");

<img src=images/cfo_estim_plot.png width='' height='' > </img>